In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pickle


In [20]:
from google.colab import files
uploaded = files.upload()

df = pd.read_csv("creditcard.csv")
df.head()


Saving creditcard.csv to creditcard (2).csv


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [21]:
df = df.drop(['Time'], axis=1)
scaler = StandardScaler()
df['Amount'] = scaler.fit_transform(df[['Amount']])
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

print("Data preprocessed and split.")


Data preprocessed and split.


In [22]:
rf_model = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',
    random_state=42
)

rf_model.fit(X_train, y_train)

print("Model trained.")


Model trained.


In [23]:
y_pred = rf_model.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))


Confusion Matrix:
[[56861     3]
 [   24    74]]

Classification Report:
              precision    recall  f1-score   support

           0     0.9996    0.9999    0.9998     56864
           1     0.9610    0.7551    0.8457        98

    accuracy                         0.9995     56962
   macro avg     0.9803    0.8775    0.9227     56962
weighted avg     0.9995    0.9995    0.9995     56962



In [24]:
def predict_by_index():
    try:
        index = int(input("Enter transaction index (0 to {}): ".format(len(df) - 1)))
        if index < 0 or index >= len(df):
            print("Index out of range.")
            return
    except ValueError:
        print("Invalid input. Please enter a valid number.")
        return

    print(f"\nTesting transaction at index {index}...\n")

    sample = df.drop("Class", axis=1).iloc[index].copy()
    actual = df['Class'].iloc[index]

    original_amount = sample['Amount']
    scaled_amount = scaler.transform([[original_amount]])[0][0]
    sample['Amount'] = scaled_amount

    input_data = pd.DataFrame([sample.values], columns=X_train.columns)

    # Predict
    prediction = rf_model.predict(input_data)[0]

    print("Original Transaction Amount: ₹", round(original_amount, 2))
    print("Model Prediction:", "FRAUD" if prediction == 1 else "NOT FRAUD")
    print("Actual Label from Dataset:", "FRAUD" if actual == 1 else "NOT FRAUD")



In [25]:
predict_by_index()


Enter transaction index (0 to 284806): 541

Testing transaction at index 541...

Original Transaction Amount: ₹ -0.35
Model Prediction: FRAUD
Actual Label from Dataset: FRAUD


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [27]:
with open("fraud_model.pkl", "wb") as f:
    pickle.dump(rf_model, f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("Model and scaler saved.")


Model and scaler saved.


In [28]:
from google.colab import files

files.download("fraud_model.pkl")
files.download("scaler.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>